In [ ]:
from models import getModel 
from dataloader import data_pipeline #, data_pipeline_with_augmentation
from tensorflow.keras.applications.efficientnet import EfficientNetB5
from tensorflow.keras.applications.resnet_v2 import ResNet50V2
from tensorflow.keras.applications.efficientnet import EfficientNetB7 
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.callbacks import ModelCheckpoint, LearningRateScheduler, ReduceLROnPlateau, CSVLogger, EarlyStopping 
from tensorflow.keras.models import load_model 
from tensorflow.keras.optimizers import Adam

import pandas as pd
import pickle, os, shutil
from IPython.display import clear_output 

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True) 

In [ ]:
datasets = [
    'classification',
    'classificationWithGAN',
] 

In [ ]:
import subprocess

for dataset in datasets:
    subprocess.Popen(f"unzip 'drive/MyDrive/{dataset}.zip'", shell=True) 

In [ ]:
preTrained = {
    'VGG16' : dict(
      modelFunction = VGG16,
      learningRate = 7e-5, 
      batchSize = 8,
      pool = 'max',
      dropout = [0.6, 0.5],
      trainbatchnorm = False,
      denseSize = [512, 32],
      epochs = 14
    ),
    'ResNet50V2' : dict(
      modelFunction = ResNet50V2,
      learningRate = 7e-5, 
      batchSize = 8,
      pool = 'avg',
      dropout = [0.5, 0.4],
      trainbatchnorm = True,
      denseSize = [512, 32],
      epochs = 12
    ),
    'EfficientNetB5' : dict(
      modelFunction = EfficientNetB5,
      learningRate = 7e-5, 
      batchSize = 8,
      pool = 'max',
      dropout = [0.6, 0.4],
      trainbatchnorm = False,
      denseSize = [512, 256],
      epochs = 10
    ),
    'EfficientNetB7' : dict(
      modelFunction = EfficientNetB7,
      learningRate = 7e-5, 
      batchSize = 8,
      pool = 'max',
      dropout = [0.6, 0.3],
      trainbatchnorm = False,
      denseSize = [512, 256],
      epochs = 4
    ),
} 

In [ ]:
!rm -rf results 
os.mkdir('results') 
for dataset in datasets:
  os.mkdir(f'results/{dataset}') 

In [ ]:
results = pd.DataFrame()

def train():
  for preTrainedModel in preTrained.keys():
      for dataset in datasets:
        modelFunction, learningRate, batchSize, pool, dropout, trainbatchnorm, denseSize, epochs = preTrained[preTrainedModel].values()
        epochs = 10
        trainDataset, validateDataset, testDataset = data_pipeline(DIR=dataset, BATCHSIZE=batchSize) 
        model = getModel(modelFunction, learning_rate=learningRate, trainbatchnorm=trainbatchnorm, pool=pool, dropout=dropout, denseSize=denseSize)

        callbacks = [ 
            EarlyStopping(patience=4, verbose=1), 
            ReduceLROnPlateau(factor=0.5, cooldown=0, patience=3, min_lr=0.5e-6), 
            CSVLogger(f'results/{dataset}/{preTrainedModel}.csv'), 
            ModelCheckpoint(f'results/{dataset}/{preTrainedModel}.hdf5', monitor = 'val_Accuracy', verbose = 1, mode='max', save_best_only=True) 
        ]

        model.fit(trainDataset, batch_size=batchSize, epochs=epochs, verbose=True, validation_data=validateDataset, shuffle=True, callbacks=callbacks)

        model = load_model(f'results/{dataset}/{preTrainedModel}.hdf5', compile=False)
        model.compile(optimizer = Adam(learning_rate=learningRate), loss = 'binary_crossentropy', metrics = ['Accuracy', 'Recall', 'Precision', 'TrueNegatives', 'TruePositives', 'FalsePositives', 'FalseNegatives'])

        res = model.evaluate(testDataset, batch_size=batchSize) 
        result = dict()
        metrics = ['loss', 'Accuracy', 'Recall', 'Precision', 'TrueNegatives', 'TruePositives', 'FalsePositives', 'FalseNegatives']
        result = {metric:value for metric, value in zip(metrics, res)}

        result['Model'] = preTrainedModel 
        result['F1_Score'] = [(2*result['Recall']*result['Precision'])/(result['Recall']+result['Precision'])]
        result['Dataset'] = dataset
        df = pd.DataFrame(result) 
        results = pd.concat([results, df], axis=0)

        clear_output(wait=True) 

train() 

In [ ]:
results.to_csv(path_or_buf='results/testResults/Results.csv', index=False)

toLatex = results[[ 'Model', 'Dataset', 'Accuracy', 'Recall', 'Precision', 'F1_Score']]
pickle.dump(toLatex.sort_values(by=['Model'], ascending=True).to_latex(index=False), open("results/testResults/Results.txt", 'wb')) 

In [ ]:
shutil.copytree("/content/results","/content/drive/MyDrive/results", dirs_exist_ok=True) 